# GEFS Wave Global Collection

Reads the Global GEFS Wave grib collection from NODD's S3 bucket and creates a single dataset using fsspec's ReferenceFileSystem

This notebook demonstrates how to generate the reference JSON files using Kerchunk and gribberish. It was adapted from [this notebook](https://nbviewer.org/gist/peterm790/92eb1df3d58ba41d3411f8a840be2452)

In [1]:
import fsspec

Create the virtual filesystem from which to read the NODD s3 bucket. We are going to write the resulting dataset out to our local filesystem, so we leave the write system blank

In [2]:
fs_read = fsspec.filesystem('s3', anon=True, skip_instance_cache=True, use_ssl=False) # For now SSL false is solving my cert issues **shrug**
fs_write = fsspec.filesystem('')

Now, grab all of the files for the given model run

In [3]:
#https://noaa-gefs-pds.s3.amazonaws.com/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.f012.grib2
gefs_ens_member_files = fs_read.glob('s3://noaa-gefs-pds/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.*.grib2')

files = sorted(['s3://'+f for f in gefs_ens_member_files])
print(f'Read {len(files)} GEFS files')

Read 105 GEFS files


For each file, we will scan it with gribberish and extract its data to a dictionary formatted to the zarr spec. Then we will write that dictionary to a JSON file with the name formatting specified in the `make_json_name` function

In [4]:
import ujson
from gribberish.kerchunk import scan_gribberish

so = {"anon": True, "use_ssl": False}
json_dir = 'gefs_wave/'

def make_json_name(file_url, message_number): #create a unique name for each reference file
    date = file_url.split('/')[3].split('.')[1]
    name = file_url.split('/')[7].split('.')[0:5]
    return f'{json_dir}{date}_{name[0]}_{name[1]}_{name[2]}_{name[4]}_message{message_number}.json'

def gen_json(file_url):
    print(f'Processing {file_url}...')
    out = scan_gribberish(file_url, storage_options=so)   # create the reference using scan_grib
    for i, message in enumerate(out): # scan_grib outputs a list containing one reference per grib message
        out_file_name = make_json_name(file_url, i)  # get name
        with fs_write.open(out_file_name, "w") as f: 
            f.write(ujson.dumps(message)) # write to file

In [5]:
%%time
#this step is best run via a cluster
for f in files[0:1]:
    gen_json(f)

Processing s3://noaa-gefs-pds/gefs.20230706/12/wave/gridded/gefs.wave.t12z.p01.global.0p25.f000.grib2...
[{'version': 1, 'refs': {'.zgroup': '{"zarr_format":2}', '.zattrs': '{"meta":"Generated with gribberishpy"}', 'wind/.zarray': '{"chunks":[1,721,1440],"compressor":null,"dtype":"<f8","fill_value":0.0,"filters":[{"dtype":"float64","id":"gribberish","var":"wind"}],"order":"C","shape":[1,721,1440],"zarr_format":2}', 'wind/0.0.0': ['{{u}}', 0, 665361], 'wind/.zattrs': '{"_ARRAY_DIMENSIONS":["time","latitude","longitude"],"coordinates":"latitude longitude","generating_process":"ensemble forecast","long_name":"windspeed","reference_date":"2023-07-06T12:00:00+00:00","standard_name":"windspeed","unit":"ms-1"}', 'time/.zarray': '{"chunks":[1],"compressor":null,"dtype":"<M8[s]","fill_value":0,"filters":null,"order":"C","shape":[1],"zarr_format":2}', 'time/0': 'base64:wKymZAAAAAA=', 'time/.zattrs': '{"_ARRAY_DIMENSIONS":["time","latitude","longitude"],"axis":"T","long_name":"time","standard_nam